<a href="https://colab.research.google.com/github/DamnCrab/Real-ESRGAN-Inference-Demo/blob/main/Real_ESRGAN_Inference_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real-ESRGAN Inference Demo

[![arXiv](https://img.shields.io/badge/arXiv-Paper-<COLOR>.svg)](https://arxiv.org/abs/2107.10833)
[![GitHub Stars](https://img.shields.io/github/stars/xinntao/Real-ESRGAN?style=social)](https://github.com/xinntao/Real-ESRGAN)
[![download](https://img.shields.io/github/downloads/xinntao/Real-ESRGAN/total.svg)](https://github.com/xinntao/Real-ESRGAN/releases)

This is a **Practical Image Restoration Demo** of our paper [''Real-ESRGAN: Training Real-World Blind Super-Resolution with Pure Synthetic Data''](https://arxiv.org/abs/2107.10833).
We extend the powerful ESRGAN to a practical restoration application (namely, Real-ESRGAN), which is trained with pure synthetic data. <br>
The following figure shows some real-life examples.

<img src="https://raw.githubusercontent.com/xinntao/Real-ESRGAN/master/assets/teaser.jpg" width="100%">

We provide a pretrained model (*RealESRGAN_x4plus.pth*) with upsampling X4.<br>
**Note that RealESRGAN may still fail in some cases as the real-world degradations are really too complex.**<br>
Moreover, it **may not** perform well on **human faces, text**, *etc*, which will be optimized later.
<br>

You can also find a **Portable Windows/Linux/MacOS executable files for Intel/AMD/Nvidia GPU.** in our [GitHub repo](https://github.com/xinntao/Real-ESRGAN). <br>
This executable file is **portable** and includes all the binaries and models required. No CUDA or PyTorch environment is needed.<br>
This executable file is based on the wonderful [Tencent/ncnn](https://github.com/Tencent/ncnn) and [realsr-ncnn-vulkan](https://github.com/nihui/realsr-ncnn-vulkan) by [nihui](https://github.com/nihui).

# 1. Preparations
Before start, make sure that you choose
* Runtime Type = Python 3
* Hardware Accelerator = GPU

in the **Runtime** menu -> **Change runtime type**

Then, we clone the repository, set up the envrironment, and download the pre-trained model.

1.1 显卡检测和系统资源监控

In [ ]:
#检测使用的显卡
! /opt/bin/nvidia-smi
! nvcc --version

#注册https://wandb.ai/authorize 填api 获得实时监控网址
!pip install wandb
import wandb
wandb.init()

1.2 完整FFmpeg安装 （可跳过

In [14]:
%cd /content
!sudo apt install build-essential curl cmake 
!git clone https://github.com/DamnCrab/ffmpeg-build-script.git
#!git clone https://github.com/markus-perl/ffmpeg-build-script
%cd ffmpeg-build-script 
#! ./build-ffmpeg -b --enable-gpl-and-non-free 
#%cd /content
! ./build-ffmpeg --h 
! ./build-ffmpeg -b --enable-gpl-and-non-free --full-static
! ./build-ffmpeg -c  

##原项目 https://github.com/markus-perl/ffmpeg-build-script 
##使用原项目需要修改build-ffmpeg文件
##1. 首行添加 AUTOINSTALL=yes   （自动跳过确认
##2. 637行删除libwebp的部分    （不兼容

/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
curl is already the newest version (7.58.0-2ubuntu3.13).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
fatal: destination path 'ffmpeg-build-script' already exists and is not an empty directory.
/content/ffmpeg-build-script
ffmpeg-build-script v1.28

Usage: build-ffmpeg [OPTIONS]
Options:
  -h, --help                     Display usage information
      --version                  Display version information
  -b, --build                    Starts the build process
      --enable-gpl-and-non-free  Enable GPL and non-free codecs  - https://ffmpeg.org/legal.html
  -c, --cleanup                  Remove all working dirs
      --full-static              Build a full static FFmpeg binary (eg. glibc, pthreads etc...) **only Linux**
                    

In [ ]:
!ffmpeg -version

1.3 Real-ESRGAN下载

In [ ]:
# Clone Real-ESRGAN and enter the Real-ESRGAN
%cd /content
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
# !pip install -r requirements.txt
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models

# 2. Upload Images

Upload the images to be processed by Real-ESRGAN

2.1 网页上传或者Drive上传

In [ ]:
#网页上传方式
import os
from google.colab import files
import shutil

upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)



In [ ]:
#Drive上传方式 需要授权
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from google.colab import files
import shutil
upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

filename='1.mkv' #填文件名########################################################################

dst_path1 = os.path.join("/content/drive/MyDrive/", filename)

shutil.copy(dst_path1,"/content/Real-ESRGAN/upload/")




In [ ]:
# 检测输入文件FPS。
tmp_path = os.path.join("/content/Real-ESRGAN/upload/", filename)
import cv2

video = cv2.VideoCapture(tmp_path)

fps = video.get(cv2.CAP_PROP_FPS)
print("帧率: {0}".format(fps))
frame_num = video.get(cv2.CAP_PROP_FRAME_COUNT)
print("帧数: {0}".format(frame_num))

#av1编码有bug
#print(f'fps= {fps}')


In [ ]:
#更新ffmpeg  （PAA更新FFmpeg4 已废弃
#!add-apt-repository ppa:jonathonf/ffmpeg-4 -y
#!apt-get update
#!apt-get install ffmpeg

In [ ]:
%shell ffmpeg -i '/content/Real-ESRGAN/upload/{filename}' -codec copy -map 0 -f segment -segment_list out.csv -segment_frames 1000,2000,3000,4000,5000 out%03d.mkv

In [ ]:
!apt-get install nodejs-dev node-gyp libssl1.0-dev
!apt-get install npm
!$ npm install --global ffmpeg-progressbar-cli


In [ ]:
! npm install --global ffmpeg-progressbar-cli

In [ ]:
%shell ffprobe -v error -show_streams -print_format json '/content/Real-ESRGAN/upload/{filename}'  

In [ ]:
%shell ffmpeg -codecs

In [18]:
%shell ffmpeg -hwaccels -h

ffmpeg version v1.28-8-gbacad10 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --enable-nonfree --enable-gpl --enable-lv2 --enable-openssl --enable-libsvtav1 --enable-libx264 --enable-libx265 --enable-libvpx --enable-libxvid --enable-libvidstab --enable-libaom --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libmp3lame --enable-libopus --enable-libvorbis --enable-libtheora --enable-libfdk-aac --enable-libsrt --enable-cuda-nvcc --enable-cuvid --enable-nvenc --enable-cuda-llvm --nvccflags='-gencode arch=compute_52,code=sm_52' --disable-debug --disable-doc --disable-shared --enable-pthreads --enable-static --enable-small --enable-version3 --extra-cflags='-I/content/ffmpeg-build-script/workspace/include -I/content/ffmpeg-build-script/workspace/include/lilv-0 -I/usr/local/cuda/include' --extra-ldexeflags=-static --extra-ldflags='-L/content/ffmpeg-build-script/workspace/lib -L/zlib/lib -L/usr/local/cuda/lib64' --e

In [ ]:
%shell ffmpeg -codecs | grep cuda


In [50]:
#拆帧
#filename='1.mkv'
%shell rm -rf '/content/Real-ESRGAN/frame'
%shell mkdir -p '/content/Real-ESRGAN/frame'
%shell ffmpeg -hwaccel -i '/content/Real-ESRGAN/upload/{filename}' '/content/Real-ESRGAN/frame/%08d.png'


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f6206477550>> (for pre_run_cell):


Exception: ignored

ffmpeg version v1.28-7-g3707270 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --enable-nonfree --enable-gpl --enable-lv2 --enable-openssl --enable-libsvtav1 --enable-libx264 --enable-libx265 --enable-libvpx --enable-libxvid --enable-libvidstab --enable-libaom --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libmp3lame --enable-libopus --enable-libvorbis --enable-libtheora --enable-libfdk-aac --enable-libsrt --enable-cuda-nvcc --enable-cuvid --enable-nvenc --enable-cuda-llvm --enable-libnpp --nvccflags='-gencode arch=compute_52,code=sm_52' --disable-debug --disable-doc --disable-shared --enable-pthreads --enable-static --enable-small --enable-version3 --extra-cflags='-I/content/ffmpeg-build-script/workspace/include -I/content/ffmpeg-build-script/workspace/include/lilv-0 -I/usr/local/cuda/include' --extra-ldexeflags= --extra-ldflags='-L/content/ffmpeg-build-script/workspace/lib -L/zlib/lib -L/usr/local/cuda/l

CalledProcessError: ignored

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f6206477550>> (for post_run_cell):


Exception: ignored

In [ ]:
%shell ffmpeg -hwaccel

# 3. Inference


In [ ]:
#超分
# if it is out of memory, try to use the `--tile` option 爆显存调tile 现值100
!python inference_realesrgan.py --model_path experiments/pretrained_models/RealESRGAN_x4plus.pth --input frame --scale 4 --half --tile 100

流式输出内容被截断，只能显示最后 5000 行内容。
	Tile 16/220
	Tile 17/220
	Tile 18/220
	Tile 19/220
	Tile 20/220
	Tile 21/220
	Tile 22/220
	Tile 23/220
	Tile 24/220
	Tile 25/220
	Tile 26/220
	Tile 27/220
	Tile 28/220
	Tile 29/220
	Tile 30/220
	Tile 31/220
	Tile 32/220
	Tile 33/220
	Tile 34/220
	Tile 35/220
	Tile 36/220
	Tile 37/220
	Tile 38/220
	Tile 39/220
	Tile 40/220
	Tile 41/220
	Tile 42/220
	Tile 43/220
	Tile 44/220
	Tile 45/220
	Tile 46/220
	Tile 47/220
	Tile 48/220
	Tile 49/220
	Tile 50/220
	Tile 51/220
	Tile 52/220
	Tile 53/220
	Tile 54/220
	Tile 55/220
	Tile 56/220
	Tile 57/220
	Tile 58/220
	Tile 59/220
	Tile 60/220
	Tile 61/220
	Tile 62/220
	Tile 63/220
	Tile 64/220
	Tile 65/220
	Tile 66/220
	Tile 67/220
	Tile 68/220
	Tile 69/220
	Tile 70/220
	Tile 71/220
	Tile 72/220
	Tile 73/220
	Tile 74/220
	Tile 75/220
	Tile 76/220
	Tile 77/220
	Tile 78/220
	Tile 79/220
	Tile 80/220
	Tile 81/220
	Tile 82/220
	Tile 83/220
	Tile 84/220
	Tile 85/220
	Tile 86/220
	Tile 87/220
	Tile 88/220
	Tile 89/220
	Tile 90/22

# 5. Download Results


In [ ]:
%shell mkdir -p '/content/Real-ESRGAN/output'

In [ ]:
%shell ffmpeg -i '/content/Real-ESRGAN/results/%08d_out.png' -c:v libx265 -preset:v [ultrafast-placebo] -profile:v main -x265-params "crf=[0-51]:ref=4:rd=3:no-rect=1:no-amp=1:b-intra=1:rdoq-level=2:limit-tu=4:me=3:subme=5:weightb=1:no-strong-intra-smoothing=1:psy-rd=2.0:psy-rdoq=1.0:no-open-gop=1:keyint=200:min-keyint=1:rc-lookahead=20:bframes=6:aq-mode=1:aq-strength=0.8:qg-size=8:cbqpoffs=-2:crqpoffs=-2:qcomp=0.65:deblock=-1:no-sao=1:range=limited:colorprim=1:transfer=1:colormatrix=1:repeat-headers=1" -pix_fmt yuv420p /content/Real-ESRGAN/output/output.mp4

In [ ]:
'/content/Real-ESRGAN/upload/{filename}' '/content/Real-ESRGAN/frame/%08d.png'

In [ ]:
# Download the results
zip_filename = 'Real-ESRGAN_result.zip'
if os.path.exists(zip_filename):
  os.remove(zip_filename)
os.system(f"zip -r -j {zip_filename} results/*")
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>